# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,36.37,23,0,4.12,US,1728435107
1,1,laojunmiao,39.8333,97.7333,7.65,50,47,0.63,CN,1728435108
2,2,qaqortoq,60.7167,-46.0333,2.26,85,92,0.72,GL,1728435109
3,3,cabo san lucas,22.8909,-109.9124,29.17,73,40,5.66,MX,1728435094
4,4,bilibino,68.0546,166.4372,-6.05,91,100,1.45,RU,1728435111


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
%%capture --no-display

# Configure the map plot
tiles = gv.tile_sources.OSM() 

map_plot = tiles * city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='City',
    title='City Humidity Map',
    hover_cols=['City', 'Humidity'],
    frame_height=400,
    frame_width=600
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[city_data_df['Max Temp'] < 30]
ideal_city_data_df = ideal_city_data_df[ideal_city_data_df['Max Temp'] > 20]
ideal_city_data_df = ideal_city_data_df[ideal_city_data_df['Humidity'] < 50]
ideal_city_data_df = ideal_city_data_df[ideal_city_data_df['Cloudiness'] < 25]
ideal_city_data_df = ideal_city_data_df[ideal_city_data_df['Wind Speed'] < 10]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,bani suwayf,29.0744,31.0979,24.52,44,0,2.57,EG,1728435169
130,130,at taj,24.2049,23.2857,26.03,22,0,3.99,LY,1728435259
224,224,duncan,34.5023,-97.9578,26.32,41,0,3.60,US,1728435367
236,236,passagem franca,-6.1797,-43.7836,24.93,41,2,0.30,BR,1728435382
245,245,al bawiti,28.3492,28.8659,25.41,34,0,2.74,EG,1728435393
270,270,xinshi,31.0470,113.1410,21.09,49,7,2.11,CN,1728435421
273,273,gurupi,-11.7292,-49.0686,29.27,42,8,1.47,BR,1728435424
301,301,deming,32.2687,-107.7586,28.94,20,0,2.57,US,1728435455
306,306,campos sales,-7.0744,-40.3761,22.94,48,0,2.71,BR,1728435463
323,323,crane,31.3974,-102.3501,26.75,27,0,4.46,US,1728435481


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
54,bani suwayf,EG,29.0744,31.0979,44,None
130,at taj,LY,24.2049,23.2857,22,None
224,duncan,US,34.5023,-97.9578,41,None
236,passagem franca,BR,-6.1797,-43.7836,41,None
245,al bawiti,EG,28.3492,28.8659,34,None
270,xinshi,CN,31.0470,113.1410,49,None
273,gurupi,BR,-11.7292,-49.0686,42,None
301,deming,US,32.2687,-107.7586,20,None
306,campos sales,BR,-7.0744,-40.3761,48,None
323,crane,US,31.3974,-102.3501,27,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bani suwayf - nearest hotel: قاعة المؤتمرات
at taj - nearest hotel: No hotel found
duncan - nearest hotel: Quality Inn
passagem franca - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
xinshi - nearest hotel: Jinshan International Hotel
gurupi - nearest hotel: Via Norte
deming - nearest hotel: No hotel found
campos sales - nearest hotel: Hotel do Município de Campos Sales
crane - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
tamanrasset - nearest hotel: فندق الأمان
sao raimundo nonato - nearest hotel: Pousada Lelinha
sakakah - nearest hotel: فندق النزل
brak - nearest hotel: فندق براك السياحي
upington - nearest hotel: Die Eiland Holiday Ressort
okara - nearest hotel: No hotel found
aswan - nearest hotel: Oscar Hotel
port augusta - nearest hotel: Hotel Commonwealth
arar - nearest hotel: دار أوتيل Dar Otel
dickinson - nearest hotel: Days Inn by Wyndham Dickinson TX
hwange - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
54,bani suwayf,EG,29.0744,31.0979,44,قاعة المؤتمرات
130,at taj,LY,24.2049,23.2857,22,No hotel found
224,duncan,US,34.5023,-97.9578,41,Quality Inn
236,passagem franca,BR,-6.1797,-43.7836,41,No hotel found
245,al bawiti,EG,28.3492,28.8659,34,Old Oasis Hotel
270,xinshi,CN,31.0470,113.1410,49,Jinshan International Hotel
273,gurupi,BR,-11.7292,-49.0686,42,Via Norte
301,deming,US,32.2687,-107.7586,20,No hotel found
306,campos sales,BR,-7.0744,-40.3761,48,Hotel do Município de Campos Sales
323,crane,US,31.3974,-102.3501,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
tiles = gv.tile_sources.OSM()

map_plot = tiles * hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size=50,
    color='Hotel Name',
    title='Hotels Map',
    hover_cols=['City', 'Hotel Name', 'Country'],
    frame_height=400,
    frame_width=600
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)